# Soccer Prediction

An endeavor to predict soccer using the previous events and experiments.


In [2]:
import json

import pandas as pd
import matplotlib

from utils import apply_time_limits


## Inspect the Events Collection
First of all lets take a look at what we have in the events collection which seems to be quite informative.

In [3]:
df = pd.read_json('../data/events_World_Cup.json', orient='records')
df.head()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId
0,8,Pass,1.656214,258612104,2057954,1H,122671,"[{'x': 50, 'y': 50}, {'x': 35, 'y': 53}]",85,Simple pass,[{'id': 1801}],16521
1,8,Pass,4.487814,258612106,2057954,1H,139393,"[{'x': 35, 'y': 53}, {'x': 75, 'y': 19}]",83,High pass,[{'id': 1801}],16521
2,1,Duel,5.937411,258612077,2057954,1H,103668,"[{'x': 25, 'y': 81}, {'x': 37, 'y': 83}]",10,Air duel,"[{'id': 703}, {'id': 1801}]",14358
3,1,Duel,6.406961,258612112,2057954,1H,122940,"[{'x': 75, 'y': 19}, {'x': 63, 'y': 17}]",10,Air duel,"[{'id': 701}, {'id': 1802}]",16521
4,8,Pass,8.562167,258612110,2057954,1H,122847,"[{'x': 63, 'y': 17}, {'x': 71, 'y': 15}]",85,Simple pass,[{'id': 1801}],16521


## Creating New Features
Gonna try adding some new features to our new data frame.

### Time
EventSec is, unfortunately, second from the beginning of each half. We need something to start counting from the beginning of the game. Which will be time.
The time feature is going to be somehow the percentage of the game. This means that, in the beginning it will be equal to zero. At the end of the first game half, it will be 0.5 and finally, 1 at the end of the second half.

In [4]:
def normalize(x, scale=0.5, shift=0):
    x['time'] = x['eventSec'] / x['eventSec'].max() * scale + shift
    return x

first_half = df[df['matchPeriod'] == '1H'].groupby('matchId').apply(normalize)
second_half = df[df['matchPeriod'] == '2H'].groupby('matchId').apply(lambda x: normalize(x, shift=0.5))

df = pd.concat([first_half, second_half])

df.tail()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId,time
101754,8,Pass,2978.301867,263885652,2058017,2H,3476,"[{'x': 46, 'y': 20}, {'x': 64, 'y': 6}]",85,Simple pass,[{'id': 1801}],9598,0.996028
101755,7,Others on the ball,2979.084611,263885653,2058017,2H,14812,"[{'x': 64, 'y': 6}, {'x': 82, 'y': 2}]",72,Touch,[],9598,0.996159
101756,8,Pass,2983.448628,263885654,2058017,2H,14812,"[{'x': 82, 'y': 2}, {'x': 100, 'y': 100}]",80,Cross,"[{'id': 401}, {'id': 801}, {'id': 1802}]",9598,0.996886
101757,4,Goalkeeper leaving line,2985.869275,263885613,2058017,2H,25381,"[{'x': 0, 'y': 0}, {'x': 18, 'y': 98}]",40,Goalkeeper leaving line,[],4418,0.997289
101758,8,Pass,3002.148765,263885618,2058017,2H,25381,"[{'x': 14, 'y': 43}, {'x': 0, 'y': 0}]",84,Launch,[{'id': 1802}],4418,1.000000


### Limit the Dataframe to the Interval

In [15]:
results = apply_time_limits(df, 0, 50)
results = results.set_index(['matchId', 'teamId'])

In [18]:
event_names = sorted(set(df['eventName']))
for e in event_names:
    col_name = 'num{}'.format(e.title().replace(' ', ''))
    
    counts_df = df[df['eventName'] == e]
    counts_df = counts_df.groupby(['matchId', 'teamId'])['id'].count().reset_index(name=col_name)
    results = results.join(counts_df.set_index(['matchId', 'teamId']))

In [19]:
results.head(100)

eventId           eventName    eventSec         id  \
matchId teamId                                                       
2057954 14358         1                Duel    5.937411  258612077   
        14358         1                Duel   21.564677  258612080   
        14358         1                Duel   39.053501  258612081   
        14358         8                Pass   40.596294  258612082   
        14358         8                Pass   43.149358  258612085   
        14358         1                Duel   45.799300  258612086   
        14358         3           Free Kick   62.091400  258612087   
        14358         8                Pass   66.167914  258612088   
        14358         8                Pass   70.223010  258612089   
        14358         8                Pass   72.535258  258612090   
        14358         1                Duel   75.950657  258612091   
        14358         3           Free Kick   93.204477  258612093   
        14358         8                Pass   94.679280  258612094   
        14358         8                Pass   96.361991  258612095   
        14358         1                Duel   97.975776  258612098   
        14358         8                Pass   99.828278  258612099   
        14358         7  Others on the ball  101.723902  258612100   
        14358         1                Duel  103.725540  258612102   
        14358         1                Duel  104.401334  258612101   
        14358         1                Duel  105.774456  258612103   
        14358         3           Free Kick  119.630485  258612105   
        14358         8                Pass  122.741318  258612107   
        14358         8                Pass  125.351291  258612108   
        14358         8                Pass  128.625569  258612109   
        14358         1                Duel  134.396858  258612114   
        14358         3           Free Kick  164.159873  258612118   
        14358         8                Pass  170.102959  258612121   
        14358         8                Pass  173.192845  258612124   
        14358         8                Pass  179.837572  258612126   
        14358         1                Duel  184.787721  258612128   
...                 ...                 ...         ...        ...   
        14358         3           Free Kick  472.475724  258612261   
        14358         8                Pass  482.460937  258612262   
        14358         8                Pass  485.163168  258612264   
        14358         8                Pass  486.784485  258612265   
        14358         8                Pass  490.173092  258612266   
        14358         8                Pass  492.304127  258612267   
        14358         1                Duel  496.678837  258612269   
        14358         1                Duel  502.492647  258612274   
        14358         8                Pass  504.816704  258612276   
        14358         8                Pass  509.822686  258612279   
        14358         1                Duel  512.206857  258612280   
        14358         7  Others on the ball  514.019459  258612296   
        14358         1                Duel  518.481307  258612297   
        14358         8                Pass  521.123673  258612300   
        14358         1                Duel  522.865791  258612301   
        14358         1                Duel  524.214132  258612306   
        14358        10                Shot  526.276996  258612307   
        14358         3           Free Kick  553.309977  258612309   
        14358         1                Duel  571.737563  260189724   
        14358         3           Free Kick  595.935623  258612328   
        14358         8                Pass  598.838586  258612329   
        14358         8                Pass  600.231652  258612331   
        14358         1                Duel  603.588116  258612333   
        14358         8                Pass  605.646500  258612334   
        14358         1   

In [57]:
set(df['eventName'])

{'Duel',
 'Foul',
 'Free Kick',
 'Goalkeeper leaving line',
 'Offside',
 'Others on the ball',
 'Pass',
 'Save attempt',
 'Shot'}

In [71]:
print(counts_df)

        eventId  eventName     eventSec         id  matchId matchPeriod  \
13            3  Free Kick    36.815886  258612125  2057954          1H   
22            3  Free Kick    62.091400  258612087  2057954          1H   
29            3  Free Kick    93.204477  258612093  2057954          1H   
45            3  Free Kick   119.630485  258612105  2057954          1H   
51            3  Free Kick   164.159873  258612118  2057954          1H   
59            3  Free Kick   209.138650  258612171  2057954          1H   
67            3  Free Kick   230.570693  258612181  2057954          1H   
72            3  Free Kick   243.141717  258612185  2057954          1H   
78            3  Free Kick   257.054411  258612193  2057954          1H   
125           3  Free Kick   351.645264  258612268  2057954          1H   
134           3  Free Kick   370.592560  258612286  2057954          1H   
141           3  Free Kick   382.226623  258612292  2057954          1H   
157           3  Free Kic